# 서포트벡터머신

---

## Part1. 분류(Classification)

### 1. 분석 데이터 준비

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [3]:
data1 = pd.read_csv('data/breast-cancer-wisconsin.csv')
data1.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [4]:
# 특성 데이터 나누기
X = data1[data1.columns[1:10]]
y = data1[['Class']]

In [6]:
# 학습, 테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, random_state=42)

In [19]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

### 2.기본모델 적용 (서포트벡터머신)
- from sklearn.svm import SVC

In [20]:
from sklearn.svm import SVC

In [21]:
model = SVC()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.984375

In [26]:
# 오차행렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print('학습데이터 오차행렬:\n', confusion_train)

학습데이터 오차행렬:
 [[329   4]
 [  4 175]]


In [23]:
# 분류예측 모델
from sklearn.metrics import classification_report
cfreport = classification_report(y_train, pred_train)
print('학습데이터 분류예측 모델:\n', cfreport)

학습데이터 분류예측 모델:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       333
           1       0.98      0.98      0.98       179

    accuracy                           0.98       512
   macro avg       0.98      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512



In [24]:
# 테스트 데이터
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9649122807017544

In [29]:
# 오차행렬
confusion_test = confusion_matrix(y_test, pred_test)
print('학습데이터 오차행렬:\n', confusion_test)

학습데이터 오차행렬:
 [[106   5]
 [  1  59]]


In [ ]:
# 분류예측 레포트
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

---

## 3.Grid Search
- SCV 하이퍼 파라미터
    - kernel : 'rbf', 'linear'
    - C, gamma

In [30]:
param = [{'kernel': ['rbf'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 
             'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}, 
            {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 
             'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}]

In [31]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(SVC(), param, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['rbf']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [34]:
print(f'Best Param : {grid_search.best_params_}')
print(f'Best Score : {round(grid_search.best_score_,2)}')
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Param : {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
Best Score : 0.97
TestSet Score: 0.9591


### 4. Random Search

In [36]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [37]:
param_distribs={'kernel': ['rbf'], 'C': randint(low=0.001, high=100), 
                'gamma': randint(low=0.001, high=100)}

In [39]:
random_search = RandomizedSearchCV(SVC(), param_distribs, cv=5, n_iter=100)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=100, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe66f940760>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe66f9401c0>,
                                        'kernel': ['rbf']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [40]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'C': 13, 'gamma': 1, 'kernel': 'rbf'}
Best Score: 0.9667
TestSet Score: 0.9532


---

## Part2. 회귀(Regression)

### 1. 분석 데이터 준비

In [43]:
data2 = pd.read_csv('data/house_price.csv')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [44]:
# 특성데이터 분류
X = data2[data2.columns[1:-1]]
y = data2[['house_value']]

In [45]:
# 학습, 테스트 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=42)

In [46]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

### 2.기본모델 적용
- from sklearn.svm import SVR

In [47]:
from sklearn.svm import SVR
model=SVR(kernel='poly')
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.45177025652823866

In [48]:
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.4699770809619139

In [49]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error 
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 70669.55244251259
테스트 데이터 RMSE: 69600.08959938577


### 3.Grid Search
- SVR 하이퍼 파라미터
    - kernel : poly
    - C, gamma

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
param_grid={'kernel': ['poly'], 'C': [0.01, 0.1, 1, 10], 
             'gamma': [0.01, 0.1, 1, 10]}

In [53]:
grid_search=GridSearchCV(SVR(kernel='poly'), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='poly',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [54]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'C': 10, 'gamma': 10, 'kernel': 'poly'}
Best Score: 0.4888
TestSet Score: 0.5092


### 4.Random Search

In [55]:
from sklearn.model_selection import RandomizedSearchCV

In [56]:
param_distribs={'kernel': ['poly'], 'C': randint(low=0.01, high=10), 
                'gamma': randint(low=0.01, high=10)}

In [57]:
random_search=RandomizedSearchCV(SVR(kernel='poly'), 
                                 param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='scale', kernel='poly',
                                 max_iter=-1, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='deprecated', n_iter=20, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe6717ac400>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe671e7c6a0>,
                                        'kernel': ['poly']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [58]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test))) 

Best Parameter: {'C': 7, 'gamma': 8, 'kernel': 'poly'}
Best Score: 0.4558
TestSet Score: 0.4815
